In [1]:
#%pip  install litellm
%load_ext autoreload
%autoreload 2

In [2]:
# Optional: ensure deps (uncomment if needed)
# %pip install -q langgraph langchain-ollama duckdb pandas pyarrow matplotlib langchain-ollama

from Agent.data_agent import SalesDataAgent

# Initialize the agent (uses default data path: data/Store_Sales_Price_Elasticity_Promotions_Data.parquet)
agent = SalesDataAgent(
    model="llama3.2:3b",       # requires Ollama running locally with this model pulled
    temperature=0.1,
    max_tokens=2000,
    streaming=True
)


In [3]:
agent.check_model()

Server is running locally
Ollama is running locally


True

In [ ]:
# Run a prompt; optionally pass a visualization goal
result = agent.run("Show me the sales in Nov 2021", visualization_goal="Sales trend for Nov 2021")

# Inspect results
print("Final tool:", result.get("tool_choice"))
print("Chart config:", result.get("chart_config"))
print("\nNumber of answer steps:", len(result.get("answer", [])))
if result.get("answer"):
    print("\nAnalysis (first 500 chars):\n", result["answer"][0][:500])

# If the last answer item is plotting code, execute it to render the chart
if result.get("chart_config") and result.get("answer"):
    chart_code = result["answer"][-1]
    exec(chart_code, globals(), locals())

Running the graph...
